In [8]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI
from langchain_core.runnables import RunnableSequence

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [12]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""
# Initialize LLM
llm = OpenAI(temperature=0)
# Create prompt template using template string
prompt_template = PromptTemplate.from_template(template=template)
# Create a runnable sequence using the prompt and the LLM
chain = prompt_template | llm

# Use the runnable sequence to make a prediction
output = chain.invoke({"input": "I get briyani for lunch today, it was a lot of carbs."})
output


'\n{\n    "sentiment": "negative",\n    "subject": "food"\n}'

### Sequential Chains

Sometimes you want to pass the output from one model to a another model. This can be done using with different SequentialChains.


In [13]:
response_template = """
You are a helpful bot that creates a 'thank you' response text. 
If customers are unsatisfied, offer them a real world assitant to talk to. 
You will get a sentiment and subject as into and evaluate. 

text: {input}
"""

review_template = PromptTemplate(input_variables=["input"], template=response_template)
review_chain = review_template | llm

In [24]:
# the chain replaces the SimpleSequential Chain
overall_chain = chain | review_chain

output = overall_chain.invoke({"input": "I ordered malaysian Rendang however it was crispy."})
output

'\nThank you for bringing this to our attention. We apologize for any dissatisfaction you may have experienced with our food. We take all feedback seriously and will work to improve our offerings. If you would like to speak with a real world assistant about your experience, please let us know and we will arrange for someone to contact you. We value your feedback and hope to have the opportunity to make it right.'

In [25]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review (for me to the restaurant) given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {review}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into Macedonian.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to macedonian: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="macedonian_translation"
)

overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "macedonian_translation"],
)

overall_chain({"dish_name": "malaysian rendang", "experience": "Traditionally, the chicken is not supposed to be crispy!"})

{'dish_name': 'malaysian rendang',
 'experience': 'Traditionally, the chicken is not supposed to be crispy!',
 'review': '\n\nI recently ordered the Malaysian rendang from your restaurant and I must say, I was blown away by the flavors and authenticity of the dish. The rich and aromatic spices used in the rendang were perfectly balanced and created a burst of flavors in every bite.\n\nHowever, I did notice that the chicken in the dish was crispy, which is not traditionally how rendang is prepared. While it did not affect the taste of the dish, I wanted to bring it to your attention as it may be a concern for other customers who are looking for an authentic rendang experience.\n\nDespite this small discrepancy, I thoroughly enjoyed my meal and would highly recommend your restaurant to anyone looking for delicious Malaysian cuisine. The service was also excellent and the staff were very friendly and accommodating.\n\nI appreciate the effort put into creating such a delicious dish and I w

In [ ]:
# # This is an LLMChain to write a review given a dish name and the experience.
# prompt_review = PromptTemplate.from_template(
#     template="You ordered {dish_name} and your experience was {experience}. Write a review: "
# )
# chain_review = prompt_review | llm

# # This is an LLMChain to write a follow-up comment given the restaurant review.
# prompt_comment = PromptTemplate.from_template(
#     template="Given the restaurant review: {{review}}, write a follow-up comment: "
# )
# chain_comment = prompt_comment | llm

# # This is an LLMChain to summarize a review.
# prompt_summary = PromptTemplate.from_template(
#     template="Summarise the review in one short sentence: \n\n {{review}}"
# )
# chain_summary = prompt_summary | llm

# # This is an LLMChain to translate a summary into German.
# prompt_translation = PromptTemplate.from_template(
#     template="Translate the summary to macedonian: \n\n {summary}"
# )
# chain_translation = prompt_translation | llm

# # Chain them together sequentially
# overall_chain = chain_review | chain_comment | chain_summary | chain_translation

# # Run the overall chain with the input data
# result = overall_chain.invoke({
#     "dish_name": "malaysian rendang",
#     "experience": "It was crispy!"
# })
# result


Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [26]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [27]:
positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [31]:
# destination chains
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

llm = OpenAI()

# destinations where the input text eventually will be processed
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # creates a chain by combining the prompt template with an LLM
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain # stores the chain in the dict, using name as the key
destination_chains

{'positive': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x00000258CDC5DDF0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x00000258CDC5E330>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'neutral': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring any positive or negative aspects. Here is the text:\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x00000258CDC5DDF0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x00000258CDC5E330>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'negative':

In [37]:
# creates list of strings of dictionary, combine name and description of the prompt
# e.g. ["positive : Good for analyzing positive sentiments",...]
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

# Joins all the strings in the destinations into a single string, remove list
# each item is separated by a newline character
destinations_str = "\n".join(destinations)

# format the template by inserting the individual destinations into it
# give access to router in selecting the appropriate chain (positive, neutral, negative)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

# creates PromptTemplate object using the formatted router_template
# uses RouterOutputParser to parse the output
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser())

# creates an object to DECIDE which destination chain to use based on the input
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# creates object which combines router chain, destination chains, default chain (if can't decide)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True
)

chain.run("I ordered korean chicken.")



> Entering new MultiPromptChain chain...
neutral: {'input': 'I ordered korean chicken.'}
> Finished chain.


'\n\nBased on the text provided, it can be inferred that the person ordered Korean chicken. This statement does not provide any positive or negative opinions about the dish, just a neutral fact.'

In [38]:
"""
> Entering new MultiPromptChain chain...
neutral: {'input': 'I ordered korean chicken.'} # format from MULTI_PROMPT_ROUTER_TEMPLATE
> Finished chain.
"""

"\n> Entering new MultiPromptChain chain...\nneutral: {'input': 'I ordered korean chicken.'} # format from MULTI_PROMPT_ROUTER_TEMPLATE\n> Finished chain.\n"

In [35]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations

['positive: Good for analyzing positive sentiments',
 'neutral: Good for analyzing neutral sentiments',
 'negative: Good for analyzing negative sentiments']

In [36]:
destinations_str = "\n".join(destinations)
destinations_str

'positive: Good for analyzing positive sentiments\nneutral: Good for analyzing neutral sentiments\nnegative: Good for analyzing negative sentiments'